# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [30]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

import re
import numpy as np
import pandas as pd
import pickle

from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM InsertTableName", engine)
X = df['message']
Y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", str(text))

    words = word_tokenize(text)
    
    words = [w for w in words if w not in stopwords.words("english")]
    
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for words in words:
        clean_tok = lemmatizer.lemmatize(words).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42) 
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred_test = pipeline.predict(X_test)
print(classification_report(y_test, y_pred_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.84      0.91      0.88      5992
               request       0.80      0.42      0.55      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.75      0.59      0.66      3219
          medical_help       0.47      0.07      0.13       638
      medical_products       0.75      0.11      0.20       418
     search_and_rescue       0.57      0.04      0.08       192
              security       0.00      0.00      0.00       144
              military       0.56      0.09      0.15       245
           child_alone       0.00      0.00      0.00         0
                 water       0.80      0.31      0.44       500
                  food       0.84      0.38      0.52       878
               shelter       0.81      0.29      0.42       705
              clothing       0.92      0.10      0.19       115
                 money       0.55      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

#parameters = {'clf__estimator__max_depth': [2, 5, None],
    # 'clf__estimator__min_samples_leaf':[1, 5, 10],
    #'clf__estimator__criterion': ['gini', 'entropy'] #,
    # 'clf__estimator__n_estimators': [10, 20, 50]
    #}

parameters = {
                'clf__estimator__n_estimators': [5],
                'clf__estimator__min_samples_split': [2, 3, 4],
                'clf__estimator__max_depth': [10]
             }
cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=2, verbose=3)

# cv = GridSearchCV(pipeline, param_grid=parameters)  

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42) 
cv.fit(X_train, y_train)

Fitting 2 folds for each of 3 candidates, totalling 6 fits
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5, score=0.1944202266782912, total= 1.7min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=5, score=0.18223433242506812, total= 1.7min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.1min remaining:    0.0s


[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=5, score=0.1853748910200523, total= 1.7min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=3, clf__estimator__n_estimators=5, score=0.19444141689373298, total= 1.7min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5, score=0.1918047079337402, total= 1.7min
[CV] clf__estimator__max_depth=10, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5 
[CV]  clf__estimator__max_depth=10, clf__estimator__min_samples_split=4, clf__estimator__n_estimators=5, score=0.18321525885558584, total= 1.7min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 15.2min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [5], 'clf__estimator__min_samples_split': [2, 3, 4], 'clf__estimator__max_depth': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [13]:
y_pred_test = cv.predict(X_test)
print(classification_report(y_test, y_pred_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      5992
               request       0.96      0.02      0.03      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.72      0.29      0.41      3219
          medical_help       0.00      0.00      0.00       638
      medical_products       1.00      0.00      0.01       418
     search_and_rescue       0.00      0.00      0.00       192
              security       0.00      0.00      0.00       144
              military       0.00      0.00      0.00       245
           child_alone       0.00      0.00      0.00         0
                 water       1.00      0.00      0.01       500
                  food       0.00      0.00      0.00       878
               shelter       1.00      0.00      0.00       705
              clothing       0.67      0.02      0.03       115
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
# Try and AdaBoostClassifier with some base parameters 

pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

parameters = {
                'clf__estimator__n_estimators': [5],
             }
cv2 = GridSearchCV(estimator=pipeline2, param_grid=parameters, cv=2, verbose=3)

# cv = GridSearchCV(pipeline, param_grid=parameters)  

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42) 
cv2.fit(X_train, y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] clf__estimator__n_estimators=5 ..................................
[CV]  clf__estimator__n_estimators=5, score=0.20717088055797733, total= 1.7min
[CV] clf__estimator__n_estimators=5 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=5, score=0.19509536784741144, total= 1.8min


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.2min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [18]:
y_pred_test = cv2.predict(X_test)
print(classification_report(y_test, y_pred_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.76      1.00      0.86      5992
               request       0.83      0.34      0.49      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.67      0.52      0.59      3219
          medical_help       0.59      0.17      0.26       638
      medical_products       0.67      0.21      0.31       418
     search_and_rescue       0.60      0.23      0.34       192
              security       0.20      0.02      0.04       144
              military       0.58      0.06      0.10       245
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.45      0.59       500
                  food       0.80      0.77      0.78       878
               shelter       0.82      0.47      0.60       705
              clothing       0.71      0.53      0.61       115
                 money       0.48      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [23]:
# Try a DecisionTreeClassifier to see if we have different results

pipeline3 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))])

parameters = {
                'clf__estimator__max_depth': [5]
             }
cv3 = GridSearchCV(estimator=pipeline3, param_grid=parameters, cv=2, verbose=3)

# cv = GridSearchCV(pipeline, param_grid=parameters)  

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42) 
cv3.fit(X_train, y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] clf__estimator__max_depth=5 .....................................
[CV]  clf__estimator__max_depth=5, score=0.20597210113339146, total= 1.7min
[CV] clf__estimator__max_depth=5 .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__max_depth=5, score=0.21831062670299728, total= 1.7min


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  5.1min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__max_depth': [5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [25]:
y_pred_test = cv3.predict(X_test)
print(classification_report(y_test, y_pred_test, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.79      0.97      0.87      5992
               request       0.83      0.38      0.52      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.68      0.51      0.58      3219
          medical_help       0.60      0.20      0.30       638
      medical_products       0.69      0.33      0.44       418
     search_and_rescue       0.59      0.23      0.33       192
              security       0.14      0.01      0.01       144
              military       0.49      0.21      0.29       245
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.56      0.65       500
                  food       0.80      0.79      0.80       878
               shelter       0.77      0.57      0.66       705
              clothing       0.73      0.44      0.55       115
                 money       0.58      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [25]:
pickle.dump(cv,open('classifier.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.